In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1212941825077735812, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 615534763411529830
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
import sys
import time
import math
import os
import pandas as pd
import numpy as np

import keras
from keras import layers, metrics, models, optimizers
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 16
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)
epochs = 50

In [6]:
train_dir = 'data_augmented/cvc300_train/'
validation_dir = 'data_augmented/cvc300_validation/'
test_dir = 'data_augmented/cvc300_test/'

image_folder = 'images'
masks_folder = 'masks'

In [7]:
"""
https://www.kaggle.com/kmader/keras-linknet
"""
from keras.models import Model
from keras.layers import Input, Conv2D, Deconv2D, MaxPool2D, concatenate, AvgPool2D
from keras.layers import BatchNormalization, Activation
from keras.layers.core import Dropout, Lambda
from keras import backend as K
from keras.regularizers import l2
from keras.layers import add


s_c2 = lambda fc, k, s = 1, activation='elu', **kwargs: Conv2D(fc, kernel_size = (k,k), strides= (s,s),
                                       padding = 'same', activation = activation,
                                       **kwargs)


s_d2 = lambda fc, k, s = 1, activation='elu', **kwargs: Deconv2D(fc, kernel_size=(k,k), strides=(s,s), 
                                                       padding = 'same', activation=activation,
                                                       **kwargs)


c2 = lambda fc, k, s = 1, **kwargs: lambda x: Activation('elu')(BatchNormalization()(
    Conv2D(fc, kernel_size = (k,k), strides= (s,s),
           padding = 'same', activation = 'linear', **kwargs)(x)))


d2 = lambda fc, k, s = 1, **kwargs: lambda x: Activation('elu')(BatchNormalization()(
    Deconv2D(fc, kernel_size=(k,k), strides=(s,s), 
             padding = 'same', activation='linear', **kwargs)(x)))


def LinkNet(img_size):
    start_in = Input((img_size, img_size, 3), name = 'Input')
    in_filt = c2(64, 7, 2)(start_in)
    in_mp = MaxPool2D((3,3), strides = (2,2), padding = 'same')(in_filt)

    enc1 = enc_block(64, 64)(in_mp)
    enc2 = enc_block(64, 128)(enc1)

    dec2 = dec_block(64, 128)(enc2)
    dec2_cat = _shortcut(enc1, dec2)
    dec1 = dec_block(64, 64)(dec2_cat)

    last_out = _shortcut(dec1, in_mp)
    
    out_upconv = d2(32, 3, 2)(last_out)
    out_conv = c2(32, 3)(out_upconv)
    out = s_d2(1, 2, 2, activation = 'sigmoid')(out_conv)

    model = Model(inputs = [start_in], outputs = [out])

    return model


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[1] / residual_shape[1]))
    stride_height = int(round(input_shape[2] / residual_shape[2]))
    equal_channels = input_shape[3] == residual_shape[3]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[3],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def enc_block(m, n):
    def block_func(x):
        cx = c2(n, 3)(c2(n, 3, 2)(x))
        cs1 = concatenate([AvgPool2D((2,2))(x), 
                           cx])
        cs2 = c2(n, 3)(c2(n, 3)(cs1))
        return concatenate([cs2, cs1])
    return block_func


def dec_block(m, n):
    def block_func(x):
        cx1 = c2(m//4, 1)(x)
        cx2 = d2(m//4, 3, 2)(cx1)
        return Dropout(0.1)(c2(n, 1)(cx2))
    return block_func

In [8]:
seed = 1142
data_gen_args = dict(rescale=1. / 255, featurewise_center=True, featurewise_std_normalization=True)
image_datagen = ImageDataGenerator(**data_gen_args)
masks_datagen = ImageDataGenerator(**data_gen_args)

test_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    train_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

mask_generator = masks_datagen.flow_from_directory(
    train_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_image_generator = image_datagen.flow_from_directory(
    validation_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

validation_mask_generator = masks_datagen.flow_from_directory(
    validation_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_image_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [image_folder],
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

test_mask_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = [masks_folder],
    color_mode='grayscale',
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    seed=seed)

Found 26112 images belonging to 1 classes.
Found 26112 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 8704 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 28 images belonging to 1 classes.


In [9]:
train_generator = zip(image_generator, mask_generator)
validation_generator = zip(validation_image_generator, validation_mask_generator)
test_generator = zip(test_image_generator, test_mask_generator)

In [10]:
nb_train_samples = len(image_generator.filenames)
nb_validation_samples = len(validation_image_generator.filenames)
nb_test_samples = len(test_image_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

nb_train_samples: 26112
nb_validation_samples: 8704
nb_test_samples: 28

predict_size_train: 1632
predict_size_validation: 544
predict_size_test: 2


In [11]:
smooth = 1.
from keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def Specificity(y_true, y_pred):
    true_negatives = K.abs(y_pred)- K.abs(y_true)
    return ((true_negatives+smooth)/(y_pred+ smooth))

def Sensitivity(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return ((y_pred+smooth)/ (y_true+smooth))

def Jaccard_index(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return ((intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

In [12]:
model = LinkNet(img_size = 256)
model.summary()
adam_opt = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer= adam_opt, loss=dice_coef_loss, 
              metrics=[dice_coef, 'acc','mse', Jaccard_index, Specificity, Sensitivity, precision, recall, f1score])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 9472        Input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 128, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [13]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples//batch_size)

Epoch 1/50
1632/1632 [==============================] - 382s 234ms/step - loss: -0.1139 - dice_coef: 0.1376 - acc: 0.5349 - mean_squared_error: 0.2428 - Jaccard_index: 0.0743 - Specificity: 0.9733 - Sensitivity: 1.4329 - precision: 0.1154 - recall: 0.8062 - f1score: 0.2002 - val_loss: -0.1752 - val_dice_coef: 0.1976 - val_acc: 0.6533 - val_mean_squared_error: 0.1990 - val_Jaccard_index: 0.1100 - val_Specificity: 0.9686 - val_Sensitivity: 1.3691 - val_precision: 0.2055 - val_recall: 0.8781 - val_f1score: 0.3311
Epoch 2/50
1632/1632 [==============================] - 383s 235ms/step - loss: -0.1638 - dice_coef: 0.1854 - acc: 0.6690 - mean_squared_error: 0.1943 - Jaccard_index: 0.1025 - Specificity: 0.9754 - Sensitivity: 1.3659 - precision: 0.1999 - recall: 0.9523 - f1score: 0.3286 - val_loss: -0.2047 - val_dice_coef: 0.2254 - val_acc: 0.7352 - val_mean_squared_error: 0.1365 - val_Jaccard_index: 0.1274 - val_Specificity: 0.9676 - val_Sensitivity: 1.2943 - val_precision: 0.3071 - val_recal

1632/1632 [==============================] - 397s 243ms/step - loss: -0.4006 - dice_coef: 0.4157 - acc: 0.8023 - mean_squared_error: 0.0497 - Jaccard_index: 0.2640 - Specificity: 0.9756 - Sensitivity: 1.1159 - precision: 0.5246 - recall: 0.9627 - f1score: 0.6759 - val_loss: -0.4268 - val_dice_coef: 0.4417 - val_acc: 0.8025 - val_mean_squared_error: 0.0461 - val_Jaccard_index: 0.2846 - val_Specificity: 0.9658 - val_Sensitivity: 1.0883 - val_precision: 0.5857 - val_recall: 0.7898 - val_f1score: 0.6685ef: 0.3912 - acc: 0.7876 - mean_squared_error: 0.0554 - Jaccard_index: 0.2444 - Specificity: 0.9757 - Sensitivity: 1.1286 - precis - ETA: 4:14 - loss: -0.3723 - dice_coef: 0.3877 - acc: 0.7948 - mean_squared_error: 0.0555 - Jaccard_index: 0.2417 - Specificity: 0.9761 - Sensit - ETA: 4:13 - loss: -0.3738 - dice_coef: 0.3891 - acc: 0.7984 - mean_squared_error: 0.0553 - Jaccard_index: 0.2429 - Specificity: 0.9760 - Sensitivity: 1.1287 - precision: 0.4952 - recall: 0.9653 - f1score:  - ETA: 4:12

1632/1632 [==============================] - 398s 244ms/step - loss: -0.5622 - dice_coef: 0.5763 - acc: 0.8238 - mean_squared_error: 0.0225 - Jaccard_index: 0.4073 - Specificity: 0.9751 - Sensitivity: 1.0545 - precision: 0.7123 - recall: 0.9472 - f1score: 0.8108 - val_loss: -0.5451 - val_dice_coef: 0.5590 - val_acc: 0.8142 - val_mean_squared_error: 0.0315 - val_Jaccard_index: 0.3899 - val_Specificity: 0.9650 - val_Sensitivity: 1.0423 - val_precision: 0.7027 - val_recall: 0.7722 - val_f1score: 0.7317ecision: 0.6976  - ETA: 2:53 - loss: -0.5471 - dice_coef: 0.5613 - acc: 0.8231 - mea - ETA: 2:21 - loss: -0.5482 - dice_coef: 0.5623 - acc: 0.8243 - mean_squared_error: 0.0238 - Jaccard_index: 0.3935 - Specificity: 0.9753 - Sensitivity: 1.0582 - precision: 0.6979 - recall: 0.9486 - f1score:  - ETA: 2:21 - loss: -0.5480 - - ETA: 1:45 - loss: -0.5523 - dice_coef: 0.5664 - acc: 0.8237 - mean_squared_error: 0.0235 - Jaccard_index: 0.3975 - Specificity: 0.9752 - Sensitivity: 1.0572 - precision: 0

1632/1632 [==============================] - 399s 244ms/step - loss: -0.6979 - dice_coef: 0.7113 - acc: 0.8320 - mean_squared_error: 0.0128 - Jaccard_index: 0.5542 - Specificity: 0.9747 - Sensitivity: 1.0242 - precision: 0.8304 - recall: 0.9291 - f1score: 0.8757 - val_loss: -0.6370 - val_dice_coef: 0.6504 - val_acc: 0.8227 - val_mean_squared_error: 0.0233 - val_Jaccard_index: 0.4848 - val_Specificity: 0.9634 - val_Sensitivity: 1.0139 - val_precision: 0.8226 - val_recall: 0.7244 - val_f1score: 0.7663ivity: 1.0270 - precision: 0.8175 - recall: 0.93 - ETA: 3:37 - loss: -0.6815 - dice_coef: 0.6950 - acc: 0.8324 - mean_squared_error: - ETA: 3:27 - loss: -0.6837 - dice_coef: 0. - ETA: 3:12 - loss: -0.6851 - dice_coef: 0.6986 - acc: 0.8337 - mean_squared_error: 0.0133 - Jaccard_in - ETA: 3:02 - loss: -0.6858 - dice_coef: 0.6993 - acc: 0.8328 - mean_squared_error: 0 - ETA: 2:51 - loss: -0.6877 - dice_coef: 0.7012 - acc: 0.8334 - mean_squared_error: 0.0132 - Jaccard_index: 0.5421 - Specificity:

1632/1632 [==============================] - 407s 250ms/step - loss: -0.7799 - dice_coef: 0.7928 - acc: 0.8351 - mean_squared_error: 0.0095 - Jaccard_index: 0.6582 - Specificity: 0.9745 - Sensitivity: 1.0114 - precision: 0.8899 - recall: 0.9193 - f1score: 0.9036 - val_loss: -0.6851 - val_dice_coef: 0.6980 - val_acc: 0.8254 - val_mean_squared_error: 0.0211 - val_Jaccard_index: 0.5400 - val_Specificity: 0.9627 - val_Sensitivity: 1.0024 - val_precision: 0.8827 - val_recall: 0.6999 - val_f1score: 0.7764ror: 0.0095 - Jaccard_index: 0.6474 - Specificity: 0.9750 - Sensitiv - ETA: 3:35 - loss: -0.7727 - dice_coef: 0.7857 - acc: 0.8394 - mean_squared_error: 0.0095 - Jaccard_index: 0.6485 - Specific - ETA: 3:10 - loss: -0.7751 - dice_coef: 0.7880 - acc: 0.8385 - mean_squared_error: 0.0095 - Jaccard_index: 0.6517 - Specificity: 0.9747 - Sensitivity: 1 - ETA: 3:05 - loss: -0.7754 - dice_coef: 0.7883 - acc: 0.8387 - mean_squared_erro - ETA: 2:54 - l - ETA: 2:17 - loss: -0.7763 - dice_coef: 0.7892 -

...

1632/1632 [==============================] - 396s 243ms/step - loss: -0.8676 - dice_coef: 0.8774 - acc: 0.8382 - mean_squared_error: 0.0057 - Jaccard_index: 0.7824 - Specificity: 0.9749 - Sensitivity: 1.0031 - precision: 0.9545 - recall: 0.9404 - f1score: 0.9472 - val_loss: -0.6969 - val_dice_coef: 0.7067 - val_acc: 0.8266 - val_mean_squared_error: 0.0216 - val_Jaccard_index: 0.5518 - val_Specificity: 0.9609 - val_Sensitivity: 0.9934 - val_precision: 0.9361 - val_recall: 0.6425 - val_f1score: 0.7569
Epoch 37/50
1632/1632 [==============================] - 395s 242ms/step - loss: -0.8688 - dice_coef: 0.8786 - acc: 0.8383 - mean_squared_error: 0.0055 - Jaccard_index: 0.7843 - Specificity: 0.9749 - Sensitivity: 1.0031 - precision: 0.9557 - recall: 0.9421 - f1score: 0.9487 - val_loss: -0.7217 - val_dice_coef: 0.7313 - val_acc: 0.8268 - val_mean_squared_error: 0.0202 - val_Jaccard_index: 0.5806 - val_Specificity: 0.9621 - val_Sensitivity: 0.9956 - val_precision: 0.9243 - val_recall: 0.6871 

1632/1632 [==============================] - 399s 245ms/step - loss: -0.8721 - dice_coef: 0.8813 - acc: 0.8384 - mean_squared_error: 0.0053 - Jaccard_index: 0.7886 - Specificity: 0.9750 - Sensitivity: 1.0031 - precision: 0.9585 - recall: 0.9450 - f1score: 0.9516 - val_loss: -0.7253 - val_dice_coef: 0.7344 - val_acc: 0.8268 - val_mean_squared_error: 0.0200 - val_Jaccard_index: 0.5847 - val_Specificity: 0.9622 - val_Sensitivity: 0.9958 - val_precision: 0.9243 - val_recall: 0.6919 - val_f1score: 0.7874ecificity: 0.9750 - Sensitivity: 1.0031 - precision: 0.9584 - recall: 0.9451 - ETA: 3:08 - loss: -0.8722 - dice_coef: 0.8815 - acc: 0.8434 - - ETA: 2:54 - loss: -0.8722 - dice_coef: 0.88 - ETA: 2:39 - loss: -0.8720 - dice_coef: 0.8813 - acc: 0.8421 - mean_squared_error: 0.0053 - Jaccard_index: 0.7885 - Specificity: 0.9752 - Sensitivity: 1.0031 - precisio - ETA: 2:36 - loss: -0.8721 - dice_coef: 0.8814 - acc: 0.8417 - mean_squared_error: 0.0053 - Jaccard_index: 0.7888 - Specificity: 0.9751 - 

1632/1632 [==============================] - 402s 246ms/step - loss: -0.8753 - dice_coef: 0.8841 - acc: 0.8386 - mean_squared_error: 0.0051 - Jaccard_index: 0.7931 - Specificity: 0.9750 - Sensitivity: 1.0030 - precision: 0.9612 - recall: 0.9482 - f1score: 0.9545 - val_loss: -0.7287 - val_dice_coef: 0.7374 - val_acc: 0.8265 - val_mean_squared_error: 0.0200 - val_Jaccard_index: 0.5883 - val_Specificity: 0.9625 - val_Sensitivity: 0.9967 - val_precision: 0.9140 - val_recall: 0.7048 - val_f1score: 0.79198763 - dice_coef: 0.8852 - acc: 0.8364 - mean_squared_error: 0.0051 - Jaccard_index: 0.7947  - ETA: 3:10 - loss: -0.8758 - dic - ETA: 1:55 - loss: -0.8751 - dice_coef: 0.8840 - acc: 0.8357 - mean_squared_error: 0.0051 - Jaccard_index: 0.7929 - Specificity: 0.9750 - Sensitivity: 1.0030 - precision: 0.9609 - recall: 0.9483 - f1sco - ETA: 1:54 - loss: -0.8751 - dice_coef: 0.8839 - acc: 0.8357 - mean_squared_er - E - ETA: 1:23 - loss: -0.8752 - dice_coef: 0.8840 - acc: 0.8357 - mean_squared_erro

1632/1632 [==============================] - 399s 245ms/step - loss: -0.8786 - dice_coef: 0.8868 - acc: 0.8387 - mean_squared_error: 0.0048 - Jaccard_index: 0.7975 - Specificity: 0.9751 - Sensitivity: 1.0030 - precision: 0.9638 - recall: 0.9519 - f1score: 0.9577 - val_loss: -0.7234 - val_dice_coef: 0.7316 - val_acc: 0.8267 - val_mean_squared_error: 0.0202 - val_Jaccard_index: 0.5818 - val_Specificity: 0.9622 - val_Sensitivity: 0.9958 - val_precision: 0.9218 - val_recall: 0.6908 - val_f1score: 0.78528 - Jaccard_index: 0.7964 - Sp - ETA: 3:21 - loss: -0.8775 - dice_coef: 0.8858 - acc: 0.8410 - mean_squared_error: 0.0048 - Jaccard_index: 0.7958 - Specificity: 0.9753 - Sensitivity: 1.0030 - precision: 0.9634 -  - ETA: 3:19 - loss: - ETA: 3:01 - loss: -0.8771 - dice_coef: 0.8854 - acc: 0.8378 - - ETA: 2:48 - loss: -0.8776 - dice_coef: 0.8858 - acc:  - ETA: 2:35 - loss: -0.8772 - dice_coef: 0.8854 - acc: 0.8361 - mean_squared_error: 0.0049 - Jaccard_in - ETA: 1:27 - loss: -0.8784 - dice_coef

1632/1632 [==============================] - 404s 248ms/step - loss: -0.8804 - dice_coef: 0.8882 - acc: 0.8388 - mean_squared_error: 0.0047 - Jaccard_index: 0.7999 - Specificity: 0.9752 - Sensitivity: 1.0030 - precision: 0.9652 - recall: 0.9542 - f1score: 0.9596 - val_loss: -0.7355 - val_dice_coef: 0.7432 - val_acc: 0.8268 - val_mean_squared_error: 0.0196 - val_Jaccard_index: 0.5958 - val_Specificity: 0.9628 - val_Sensitivity: 0.9971 - val_precision: 0.9147 - val_recall: 0.7135 - val_f1score: 0.7980ificity: - ETA: 3:42 - loss: -0.8808 - dice_coef: 0.8886 - acc: 0.8386 - mean_squared_error: 0.0046 - Jaccard_index: 0.8005 - Specificity: 0.9755 - Sensitivit - ETA: 3:37 - loss: -0.8809 - dice_coef: 0.8887 - acc: 0.8398 - mean_squared_error: 0.0047 - Jaccard_index: 0.8006 - Specif - ETA: 3:30 - loss: -0.8807 - dice_coef: 0.8886 - acc: 0.8405 - mean_squared_error: 0.0046 - Jaccard_index: 0.8004 - Specificity: 0.9755 - Sensitivity: 1.0029 - precision: 0.9644 - reca - ETA: 3:27 - loss: -0.8806

In [14]:
score = model.evaluate_generator(validation_generator, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])

Validation Score:  -0.7249414658546448
Validation Accuracy:  0.732685718536377


In [15]:
prediction = model.predict_generator(test_generator, verbose=1, steps=nb_test_samples)

28/28 [==============================] - 5s 179ms/step


In [16]:
prediction.shape

(392, 256, 256, 1)

In [17]:
test_results = model.evaluate_generator(test_generator, steps=50)

print(model.metrics_names)
print(test_results)

['loss', 'dice_coef', 'acc', 'mean_squared_error', 'Jaccard_index', 'Specificity', 'Sensitivity', 'precision', 'recall', 'f1score']
[-0.7724999097415379, 0.7802441654886518, 0.9798227146693639, 0.01984564715198108, 0.6433794982092721, 0.9680362166677202, 0.9987403883252826, 0.8569075809206281, 0.721968252658844, 0.7803062282289778]


In [18]:
coef = np.array(history.history['Jaccard_index'])
val_coef = np.array(history.history['val_Jaccard_index'])
print("Training co-effiency    : {};\nValidation co-effiency : {}".format(coef[coef==max(coef)][0], val_coef[np.argmax(coef)]))

Training co-effiency    : 0.7998570555127135;
Validation co-effiency : 0.5957816480932867


In [ ]:
import skimage.io as io

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

os.mkdir('05.U-Net2_results_dir')

In [ ]:
saveResult('05.U-Net2_results_dir',prediction)

In [ ]:
def normalizeData(img,mask):
    mean = np.mean(img)
    std = np.std(img)
    img -= mean
    img /= std
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img,mask)

test_data_dir= "data_augmented/cvc300_test/images/"
test_data_mask_dir = "data_augmented/cvc300_test/masks/"
test_data_pred_dir = "05.U-Net2_results_dir"
img_rows, img_cols = 256, 256

In [ ]:
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

file_names = next(os.walk(test_data_dir))[2]
scores = []
for file in file_names:
    grey_img = load_img(os.path.join(test_data_dir,file), target_size=(img_rows, img_cols), grayscale=False)
    mask_img = load_img(os.path.join(test_data_mask_dir,file), target_size=(img_rows, img_cols), grayscale=True)
    img = img_to_array(grey_img)
    img_mask = img_to_array(mask_img)
    
    img, img_mask = normalizeData(img, img_mask)
    img = np.reshape(img,(1,)+img.shape)
    
    pred = model.predict([img])
    sess = tf.Session()
    score = sess.run(Jaccard_index(img_mask, pred))
    print("{} -- jaccard index: {}".format(file,score))
    scores.append([file,score])

    result_img = array_to_img(pred[0] * 255 )
    result_img.save(os.path.join(test_data_pred_dir, file.split('.')[0] + '_predict.jpg'))

with open("unet_test_result.csv", 'w') as f:
    f.write("filename, jaccard_index\n")
    for i in range(len(scores)):
        #print(scores[i])
        f.write("{},{}\n".format(scores[i][0], scores[i][1]))

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/102_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray")

In [ ]:
plt.imshow(np.squeeze(prediction[1]), cmap = "gray", vmin = 0.0, vmax= 0.09)

In [ ]:
results_ =  np.squeeze(np.multiply(prediction[1], 255))
print(results_)

plt.imshow(results_, cmap = "gray", vmin = 0.0, vmax= 0.7)
plt.show()

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('05.U-Net2_results_dir/35_predict.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coeff')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()